In [1]:
import pandas as pd
import numpy as np
import datetime
from yahoo_fin.stock_info import get_data,get_quote_table
import statsmodels.formula.api as smf
from pypfopt.efficient_frontier import EfficientFrontier

import json
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

In [2]:
CC_string = "ADA,ADX,AE,ANT,ARDR,ARK,BAT,BCH,BCN,BNB,BNT,BTC,BTG,BTS,CVC,DASH,DCR,DGB,DGD,DNT,DOGE,EDG,EOS,ETC,ETH,FUN,GAS,GBYTE,GNO,HC,ICX,KIN,KMD,KNC,LINK,LRC,LSK,LTC,MAID,MCO,MIOTA,MLN,MONA,MTL,NANO,NEO,NXS,OMG,PIVX,PPT,QRL,QTUM,REP,RLC,SALT,SC,SNT,STEEM,STORJ,SYS,TRX,VERI,VET,WAVES,WTC,XEM,XLM,XMR,XRP,XVG,ZEC,ZEN,ZRX"
CC_temp_list = CC_string.split(",")
CC_list = [i+'-USD' for i in CC_temp_list]

In [13]:
#Output the top 10 CC
def top10MarketCapCC():    
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest?convert=USD'
    parameters = {
        'symbol' : 'ADA,ADX,AE,ANT,ARDR,ARK,BAT,BCH,BCN,BNB,BNT,BTC,BTG,BTS,CVC,DASH,DCR,DGB,DGD,DNT,DOGE,EDG,EOS,ETC,ETH,FUN,GAS,GBYTE,GNO,HC,ICX,KIN,KMD,KNC,LINK,LRC,LSK,LTC,MAID,MCO,MIOTA,MLN,MONA,MTL,NANO,NEO,NXS,OMG,PIVX,PPT,QRL,QTUM,REP,RLC,SALT,SC,SNT,STEEM,STORJ,SYS,TRX,VERI,VET,WAVES,WTC,XEM,XLM,XMR,XRP,XVG,ZEC,ZEN,ZRX'
    }
    headers = {
        'Accepts': 'application/json',
    #Input your API key here
        'X-CMC_PRO_API_KEY': '4797f5ee-9a2d-42d1-979a-ae14643c966d',
    }
    session = Session()
    session.headers.update(headers)
    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        result = data
        d = pd.DataFrame(result)
        f = d.iloc[6:,1]
        MarketCap_rank = pd.DataFrame(columns = ['Symbol', 'MarketCap'])
        for i in range(73):
            Append = {'Symbol': f.iloc[i]['symbol'], 'MarketCap' : f.iloc[i]['quote']['USD']['market_cap']}
            MarketCap_rank = MarketCap_rank.append(Append, ignore_index=True)
        MarketCap_rank = MarketCap_rank.sort_values(by=['MarketCap'], ascending = False)
        MarketCap_rank.reset_index(drop=True, inplace=True)
        MarketCap_rank = MarketCap_rank.truncate(before=None, after=9, copy=True)
        return MarketCap_rank['Symbol'] + '-USD'
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

In [14]:
top10MarketCapCC_list = top10MarketCapCC()

In [15]:
top10MarketCapCC_list

0     BTC-USD
1     ETH-USD
2     XRP-USD
3     ADA-USD
4    LINK-USD
5     LTC-USD
6     BCH-USD
7     XLM-USD
8     BNB-USD
9    DOGE-USD
Name: Symbol, dtype: object

In [6]:
weekly_data = get_data("ADA-USD", interval = "1wk")

In [7]:
weekly_data

,open,high,low,close,adjclose,volume,ticker
2017-09-24,0.021678,0.032226,0.017354,0.024969,0.024969,50068700,ADA-USD
2017-10-01,0.024607,0.030088,0.017620,0.020477,0.020477,112121070,ADA-USD
2017-10-08,0.020344,0.038268,0.020342,0.030049,0.030049,70248060,ADA-USD
2017-10-15,0.029939,0.030855,0.024637,0.027729,0.027729,35460970,ADA-USD
2017-10-22,0.027684,0.029837,0.023766,0.028607,0.028607,25102830,ADA-USD
...,...,...,...,...,...,...,...
2021-01-04,0.205236,0.353344,0.194450,0.304895,0.304895,30627294564,ADA-USD
2021-01-11,0.304996,0.395653,0.233033,0.379852,0.379852,27415450604,ADA-USD
2021-01-18,0.379735,0.395443,0.283576,0.353881,0.353881,22674159877,ADA-USD
2021-01-25,0.353850,0.366772,0.305461,0.348131,0.348131,13911430300,ADA-USD


In [8]:
#Output market returns
def MarketReturns(CC_list):
    data = pd.DataFrame()
    end = datetime.date.today()
    start = datetime.datetime.now() - datetime.timedelta(days = 7 * 8)
    for i in CC_list:
        data[i] = get_data(i, start, end,True, '1d')['adjclose']
    data= data.fillna(value=.001)#Usd because faulty API returns NaN values
    marketData = np.log(data/ data.shift(1)).fillna(value=0.001)
    sum_row = marketData.sum(axis=1)
    return sum_row
    

In [9]:
I = MarketReturns(CC_list)

In [10]:
J = pd.DataFrame(I, columns = ['MarketReturns'])

In [16]:
top10MarketCapCC_list

0     BTC-USD
1     ETH-USD
2     XRP-USD
3     ADA-USD
4    LINK-USD
5     LTC-USD
6     BCH-USD
7     XLM-USD
8     BNB-USD
9    DOGE-USD
Name: Symbol, dtype: object

In [17]:
#Load the list of CC used in the research
end = datetime.date.today()
start = datetime.datetime.now() - datetime.timedelta(days = 7 * 8)

data = pd.DataFrame()
for i in top10MarketCapCC_list:
    data[i] = get_data(i, start, end,True, '1d')['adjclose']
data= data.fillna(value=.001)
marketData = np.log(data/ data.shift(1)).fillna(value=0.001)

In [18]:
result = pd.concat([marketData.rename(columns=lambda s: s.replace('-USD','')), J], axis=1)

In [26]:
result

,BTC,ETH,XRP,ADA,LINK,LTC,BCH,XLM,BNB,DOGE,MarketReturns
2020-12-06,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.073000
2020-12-07,-0.007966,-0.016864,-0.017769,-0.030271,-0.030969,0.000822,-0.008633,-0.041681,-0.005522,-0.013670,-0.157749
2020-12-08,-0.046418,-0.064584,-0.085854,-0.089547,-0.074181,-0.085795,-0.059303,-0.097549,-0.061120,-0.050003,-4.813535
2020-12-09,0.012625,0.033064,0.043343,0.050077,0.039367,0.008593,0.004332,0.119646,0.017682,-0.006628,0.968784
2020-12-10,-0.015695,-0.024359,-0.018393,-0.042542,-0.057941,-0.033916,-0.010209,-0.066758,-0.015517,-0.016925,-1.503521
2020-12-11,-0.011347,-0.025115,-0.047302,-0.018086,-0.022229,-0.037335,-0.025318,-0.063567,-0.012176,-0.008409,-2.166655
2020-12-12,0.040412,0.040872,-0.076174,0.035304,0.046013,0.062113,0.031816,0.050119,0.028942,0.026916,3.138844
2020-12-13,0.017854,0.036432,0.009948,0.063716,0.057650,0.066735,0.028116,0.095350,0.032736,0.028979,1.702549
2020-12-14,0.005432,-0.006213,-0.027256,0.025826,-0.008241,0.004411,0.004365,-0.029542,0.031386,-0.007708,-0.122824
2020-12-15,0.008816,0.005691,-0.057906,-0.023613,-0.009563,-0.011755,0.041926,-0.019060,-0.016191,-0.007768,0.569868


In [19]:
result.mean()
result.std()

BTC               3.277840
ETH               2.671550
XRP               1.109844
ADA               1.112787
LINK              1.901500
LTC               2.231797
BCH               2.442833
XLM               1.087859
BNB               2.020368
DOGE              0.716592
MarketReturns    97.541065
dtype: float64

In [27]:
list(result.iloc[:,0:10])

['BTC', 'ETH', 'XRP', 'ADA', 'LINK', 'LTC', 'BCH', 'XLM', 'BNB', 'DOGE']

In [20]:
Name = list(result.iloc[:,0:10])
regression = pd.DataFrame(columns=['beta', 'rse'])
for i in range(10):
    CAPM_model = smf.ols(formula = Name[i]+' ~ MarketReturns', data = result)
    CAPM_fit = CAPM_model.fit()
    beta = CAPM_fit.params['MarketReturns']
    rse = CAPM_fit.bse['MarketReturns']
    dict = {'beta': beta, 'rse': rse} 
    df = pd.DataFrame(dict, index = [Name[i]])
    regression = regression.append(df)
    
print(regression)

          beta       rse
BTC   0.033592  0.000125
ETH   0.027383  0.000077
XRP   0.011337  0.000130
ADA   0.011395  0.000073
LINK  0.019485  0.000080
LTC   0.022875  0.000066
BCH   0.025038  0.000074
XLM   0.011118  0.000119
BNB   0.020706  0.000071
DOGE  0.007023  0.000291


In [29]:
def calculateReturns(x):
    for i in range(10):
        y = str(x[i:i+1]).split(' ')[0]
        print(y)
        x[i] = result.mean()['MarketReturns'] * regression.loc[y]['beta']
    return x

In [30]:
ExpectedReturns = pd.DataFrame(index = ['returns'],columns = list(result.iloc[:,0:10]))
print(ExpectedReturns)
ExpectedReturns = ExpectedReturns.apply(lambda x: calculateReturns(x), axis=1) 
ExpectedReturns.T

         BTC  ETH  XRP  ADA LINK  LTC  BCH  XLM  BNB DOGE
returns  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
BTC
ETH
XRP
ADA
LINK
LTC
BCH
XLM
BNB
DOGE


,returns
BTC,0.017649
ETH,0.014387
XRP,0.005957
ADA,0.005987
LINK,0.010238
LTC,0.012019
BCH,0.013155
XLM,0.005841
BNB,0.010879
DOGE,0.003690


In [23]:
def calculateCovariance(x):
    for i in range(10):
        y = str(x[i:i+1]).split(' ')[0]
        if(y == x.name):
            x[i] = (result.std()['MarketReturns'] * regression.loc[y]['beta']) ** 2 + regression.loc[y]['rse']
        else:
            x[i] = (result.std()['MarketReturns'] ** 2) * (regression.loc[y]['beta'] * regression.loc[x.name]['beta'])
    return x

In [24]:
CovarianceMatrix = pd.DataFrame(index = list(result.iloc[:,0:10]),columns = list(result.iloc[:,0:10]))
CovarianceMatrix = CovarianceMatrix.apply(lambda x: calculateCovariance(x), axis=1) 
CovarianceMatrix 

,BTC,ETH,XRP,ADA,LINK,LTC,BCH,XLM,BNB,DOGE
BTC,10.736235,8.751683,3.623360,3.641998,6.227590,7.311039,8.002251,3.553328,6.617816,2.244563
ETH,8.751683,7.134127,2.953630,2.968823,5.076503,5.959691,6.523141,2.896543,5.394600,1.829685
XRP,3.623360,2.953630,1.222988,1.229148,2.101767,2.467423,2.700702,1.199223,2.233465,0.757524
ADA,3.641998,2.968823,1.229148,1.235544,2.112578,2.480115,2.714594,1.205391,2.244954,0.761420
LINK,6.227590,5.076503,2.101767,2.112578,3.612457,4.240843,4.641787,2.061144,3.838732,1.301981
LTC,7.311039,5.959691,2.467423,2.480115,4.240843,4.978712,5.449345,2.419733,4.506577,1.528494
BCH,8.002251,6.523141,2.700702,2.714594,4.641787,5.449345,5.964620,2.648503,4.932645,1.673004
XLM,3.553328,2.896543,1.199223,1.205391,2.061144,2.419733,2.648503,1.176163,2.190297,0.742882
BNB,6.617816,5.394600,2.233465,2.244954,3.838732,4.506577,4.932645,2.190297,4.079340,1.383565
DOGE,2.244563,1.829685,0.757524,0.761420,1.301981,1.528494,1.673004,0.742882,1.383565,0.469554


In [25]:
ef1 = EfficientFrontier(ExpectedReturns.iloc[0], CovarianceMatrix, weight_bounds=(0, 1))
weights = ef1.max_sharpe(risk_free_rate= 0.0)
weights = ef1.clean_weights()

weights

OrderedDict([('BTC', 0.12225),
             ('ETH', 0.16065),
             ('XRP', 0.03943),
             ('ADA', 0.07035),
             ('LINK', 0.11086),
             ('LTC', 0.15741),
             ('BCH', 0.15287),
             ('XLM', 0.04242),
             ('BNB', 0.1328),
             ('DOGE', 0.01095)])